In [2]:
!pip install opencv-python-headless
!pip install opencv-python
!pip install scipy
!pip3 install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 1.9 MB/s eta 0:00:001.9 MB/s eta 0:00:01


In [3]:
import scipy.io
import cv2
import pandas as pd
import os
import random 
import numpy as np
import json
import math

In [6]:
def parseVideoName(video_name):
    video_name = video_name.replace('.avi', '')
    first_part,second_part = video_name.split(',')
    video_type, num_targets = first_part.rsplit('_', 1)
    videos_info = second_part.split('_')
    
    parsed_data = {
        'video_name': video_name,
        'target_name': second_part,
        'video_type': video_type,
        'num_targets': int(num_targets),
        'first_video': int(videos_info[0]),
        'second_video': int(videos_info[1]),
        'first_start_frame': int(videos_info[2]),
        'second_start_frame': int(videos_info[3]),
        'first_rotation': float(videos_info[4]),
        'second_rotation': float(videos_info[5])
    }
    
    return parsed_data
    
def generateRandomColor():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def processTrialsVideos(file_path):
    df = pd.read_csv(file_path)
    parsed_rows = []
    
    for index, row in df.iterrows():
        video_name = row['videoName']
        parsed_data = parseVideoName(video_name)
        parsed_data['firstTargets'] = row['firstTargets']
        parsed_data['secondTargets'] = row['secondTargets']
        parsed_data['fullRotation'] = row['rotAngle']
        parsed_rows.append(parsed_data)
        # parsed_rows.append(1)
    return pd.DataFrame(parsed_rows)
    
def loadMatFile():
    root_folder = 'GazeMatV21'
    test_subs = ['sub11'] # 'sub21', 'sub31', 'sub41', 'sub51'

    data = {}
    
    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            mat_file_path = os.path.join(root_folder, sub_dir, 'finalSampled.mat')
            if os.path.exists(mat_file_path):
                mat_data = scipy.io.loadmat(mat_file_path)
                print(mat_data)
                data[sub_dir] = mat_data["extractedData"]
            else:
                print(f"{mat_file_path} does not exist.")
    print("Data loaded successfully.")
    return data


def loadShapeFishPoints(videoName,frame):
    json_filename = f'img{frame:03d}.json'
    json_path = os.path.join('./jsons', str(videoName), json_filename)

    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file not found: {json_path}")

    with open(json_path, 'r') as json_file:
        data = json.load(json_file)
    
    return data.get('shapes', [])

def getTargetsFromJson(shapes,targets):
    if not isinstance(targets, (list, tuple)):
        targets = [targets]
    
    target_digits = set()
    for target in targets:
        if isinstance(target, int):
            target_digits.update(str(target))
        else:
            raise ValueError("Targets should be integers or lists/tuples of integers")

    filtered_shapes = [shape for shape in shapes if shape['label'] in target_digits]
    
    return filtered_shapes

def getTotalShapeFirst(shapeInfo,baseFrame):
    firstFrame = shapeInfo['firstStart'] + baseFrame
    firstTargets = shapeInfo['firstTargets']
    shapes_video1 = loadShapeFishPoints(shapeInfo['first'], firstFrame)
    return getTargetsFromJson(shapes_video1,firstTargets)
    
def getTotalShapeSecond(shapeInfo,baseFrame):
    secondFrame = shapeInfo['secondStart'] + baseFrame
    secondTargets = shapeInfo['secondTargets']
    shapes_video2 = loadShapeFishPoints(shapeInfo['second'], secondFrame)
    merged_shapes = getTargetsFromJson(shapes_video2,secondTargets)
    return merged_shapes

def generateSubjectName(subIndex):
    number_part = subIndex[3:]
    modified_number = number_part[:-1] if len(number_part) > 1 else '0'
    return f'sub{modified_number}'

def getVideoInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # num_frames = int(mainVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height 

def createOutputVideo(video,name):
    output_gaze_videos = 'withShape'
    os.makedirs(output_gaze_videos, exist_ok=True)
    fps, width, height = getVideoInfo(video)
    
    output_gaze_videos_path = os.path.join(output_gaze_videos, name + '.avi')
    output_video = cv2.VideoWriter(output_gaze_videos_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    return output_video

def getSubjectColor(subject):
    if subject not in subject_colors:
        subject_colors[subject] = generateRandomColor()
    return subject_colors[subIndex]


def openVideo(video):
    trialVideo = cv2.VideoCapture(os.path.join('./videos', video['video_type'], video['target_name'] + '.avi'))
    
    if not trialVideo.isOpened():
        raise ValueError(f"Unable to open video file: {os.path.join('./videos', video['video_type'], video['target_name'] + '.avi')}")
    
    return trialVideo

def generateXPoints(x):
    # scale_x = 660 / 1920
    # return int(x * scale_x)
    return int(x) - 630

def generateYPoints(y):
    # scale_y = 660 / 1080
    # return int(y * scale_y)
    return int(y) - 210

def rotate_point(point, angle,center):
    angle_rad = -np.deg2rad(angle)
    M = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])
    
    translated_point = np.array(point) - np.array(center)
    rotated_point = np.dot(M, translated_point)
    rotated_point += np.array(center)
    
    return int(rotated_point[0] + 20), int(rotated_point[1] + 80)



def adjust_shape_points(shape, x_offset=90, y_offset=10):
    adjusted_points = []
    for point in shape['points']:
        adjusted_point = [point[0] - x_offset, point[1] - y_offset]
        adjusted_points.append(adjusted_point)
    shape['points'] = adjusted_points
    return shape
    
def draw_shapes(frame, shapes, typeV, color=(0, 0, 255)):
    data = {
        1: [90, 5],
        7: [25,25],
        10: [25,15]
    }
    overlay = frame.copy()
    opacity = 0.1
    for shape in shapes:
        shape = adjust_shape_points(shape,data[typeV][0],data[typeV][1])  # Adjust the points
        points = shape['points']

        if not points:
            continue
        
        np_points = np.array(points, dtype=np.int32).reshape((-1, 1, 2))

        if np_points.size == 0:
            continue
        # x, y = np_points[0][0]  # Position for the text
        # cv2.putText(frame, f"_{typeV}", (x + 20, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 225), 1, cv2.LINE_AA)
        cv2.polylines(overlay, [np_points], isClosed=True, color=color, thickness=1)
        cv2.fillPoly(overlay, [np_points], color=color)
    cv2.addWeighted(overlay, opacity, frame, 1 - opacity, 0, frame)

def rotate_frame(frame, angle, center):
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))
    return rotated_frame

# def draw_and_rotate_shapes(frame, shapes, rotation_angle, color=(0, 0, 255)):
#     # Create a blank frame for drawing shapes
#     blank_frame = np.zeros_like(frame)
#     draw_shapes(blank_frame, shapes, color)
    
#     # Rotate the blank frame with the shapes drawn on it
#     center = (blank_frame.shape[1] // 2, blank_frame.shape[0] // 2)
#     rotated_shapes_frame = rotate_frame(blank_frame, rotation_angle, center)
    
#     return rotated_shapes_frame

In [7]:
subjects = loadMatFile()
subject_colors = {}

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Oct  3 19:38:27 2024', '__version__': '1.0', '__globals__': [], 'itemsSampled': array([[array([[1017.29998779],
               [1014.20001221],
               [1014.90002441],
               [1177.5       ],
               [1180.        ],
               [1179.        ],
               [1175.59997559],
               [1187.19995117],
               [1184.90002441],
               [1182.        ],
               [1181.59997559],
               [1179.40002441],
               [1175.09997559],
               [1178.19995117],
               [1172.40002441],
               [1167.69995117],
               [1165.69995117],
               [1162.69995117],
               [1160.5       ],
               [1161.40002441],
               [1164.69995117],
               [1163.30004883],
               [1163.30004883],
               [1162.        ],
               [1163.        ],
               [1198.40002441],
               

KeyError: 'extractedData'

In [8]:
videoData = processTrialsVideos('./sub21.csv')
videoData

,video_name,target_name,video_type,num_targets,first_video,second_video,first_start_frame,second_start_frame,first_rotation,second_rotation,firstTargets,secondTargets,fullRotation
0,"NO_2,1_10_1_1_45_315",1_10_1_1_45_315,NO,2,1,10,1,1,45.0,315.0,12,0,278
1,"NO_2,1_10_119_344_45_90",1_10_119_344_45_90,NO,2,1,10,119,344,45.0,90.0,3,4,117
2,"OB_4,1_10_1_344_45_315",1_10_1_344_45_315,OB,4,1,10,1,344,45.0,315.0,3412,0,283
3,"OB_2,10_10_114_229_0_135",10_10_114_229_0_135,OB,2,10,10,114,229,0.0,135.0,14,0,170
4,"NO_4,10_10_229_344_0_315",10_10_229_344_0_315,NO,4,10,10,229,344,0.0,315.0,43,41,13
5,"OB_2,1_10_1_1_45_0",1_10_1_1_45_0,OB,2,1,10,1,1,45.0,0.0,0,14,64
6,"OC_4,1_10_119_344_0_247.5",1_10_119_344_0_247.5,OC,4,1,10,119,344,0.0,247.5,13,23,260
7,"OB_OC_4,1_10_119_114_45_225",1_10_119_114_45_225,OB_OC,4,1,10,119,114,45.0,225.0,342,1,171
8,"OC_4,1_10_119_344_45_90",1_10_119_344_45_90,OC,4,1,10,119,344,45.0,90.0,31,41,55
9,"OB_4,1_10_119_229_45_225",1_10_119_229_45_225,OB,4,1,10,119,229,45.0,225.0,342,4,123


In [9]:
videoList = []
trialNumber  = 1

for index, video in videoData.iterrows():
    try:
        
        open_video = openVideo(video)
        output_video = createOutputVideo(open_video,video.video_name)
        frame_idx = 0

        while True:
            ret, frame = open_video.read()
            if not ret:
                break  

            shapeInfo = {
                'first': video.first_video,
                'second': video.second_video,
                'firstTargets': video.firstTargets,
                'secondTargets': video.secondTargets,
                'firstStart': video.first_start_frame,
                'secondStart': video.second_start_frame
            }


            shapeFirst = getTotalShapeFirst(shapeInfo, frame_idx)
            shapeSecond = getTotalShapeSecond(shapeInfo, frame_idx)
            
            blank_frame = np.zeros_like(frame)
            blank_frame2 = np.zeros_like(frame)
            draw_shapes(blank_frame, shapeFirst,video.first_video)
            draw_shapes(blank_frame2, shapeSecond,video.second_video)

            center = (330,330)
            rotated_shapes_frame = rotate_frame(blank_frame, video.first_rotation, center)
            rotated_shapes_frame2 = rotate_frame(blank_frame2, video.second_rotation, center)

            mask = rotated_shapes_frame > 0
            frame[mask] = rotated_shapes_frame[mask]

            mask2 = rotated_shapes_frame2 > 0
            frame[mask2] = blank_frame[mask2]

            
            resized_frame = cv2.resize(frame, (660, 660))
            output_video.write(resized_frame)
            
            frame_idx += 1
        
        trialNumber += 1
        
    except Exception as e:
        print(f"An error occurred while processing video at index {index}: {e}")

    finally:
        # Release everything
        if 'open_video' in locals() and open_video.isOpened():
            open_video.release()
        if 'output_video' in locals():
            output_video.release()
        cv2.destroyAllWindows()

    